# Логистическая регрессия, метод опорных векторов, one-hot кодирование

### О задании

В этом задании вы:
- настроите метод опорных векторов
- изучите методы работы с категориальными переменными

In [10]:
%pylab inline
import pandas as pd
import numpy as np
import copy
import sklearn as sk
from sklearn.base import BaseEstimator
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


C:\Users\a9282\AppData\Roaming\Python\Python310\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


__Задание 1.__ Обучение логистической регрессии на реальных данных и оценка качества классификации.

**(5 баллов)**


Загрузим данные с конкурса [Kaggle Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) (вам нужна только обучающая выборка). Задача состоит в определении водителей, которые в ближайший год воспользуются своей автомобильной страховкой (бинарная классификация). Но для нас важна будет не сама задача, а только её данные. При этом под нужды задания мы немного модифицируем датасет.

In [11]:
import pandas as pd
data = pd.read_csv('train.csv', index_col=0)
target = data.target.values
data = data.drop('target', axis=1)

Пересемплируем выборку так, чтобы положительных и отрицательных объектов в выборке было одинаковое число. Разделим на обучающую и тестовую выборки.

In [12]:
np.random.seed(910)
mask_plus = np.random.choice(np.where(target == 1)[0], 100000, replace=True)
mask_zero = np.random.choice(np.where(target == 0)[0], 100000, replace=True)

data = pd.concat((data.iloc[mask_plus], data.iloc[mask_zero]))
target = np.hstack((target[mask_plus], target[mask_zero]))

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5)


Не забудьте отнормировать признаки (можно воспользоваться StandardScaler или сделать это вручную). Пока не будем обращать внимание на то, что некоторые признаки категориальные (этим мы займёмся позже).

In [13]:
def normalize(data):
    new_data = copy.deepcopy(data)
    for c in data.columns:
        new_data[c] = (new_data[c] - new_data[c].min()) / (new_data[c].max() - new_data[c].min())
    return new_data

Обучите логистическую регрессию с удобными для вас параметрами, примените регуляризацию. Сделайте предсказание на тестовой части выборки. Посчитайте accuracy, precision, recall и F меру

In [14]:
def train_model(X_train, X_test, y_train, y_test):
    model = LogisticRegression(max_iter=10000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f = f1_score(y_test, y_pred)
    print(f"Accuracy:", accuracy)
    print(f"Precision:", precision)
    print(f"Recall:", recall)
    print(f"F Score:", f)

In [15]:
print('Not normalized:')
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5)
train_model(X_train, X_test, y_train, y_test)
print('Normalized:')
X_train, X_test, y_train, y_test = train_test_split(normalize(data), target, test_size=0.5)
train_model(X_train, X_test, y_train, y_test)

Not normalized:
Accuracy: 0.59057
Precision: 0.5981654957064794
Recall: 0.5517331039862392
F Score: 0.5740118402297296
Normalized:
Accuracy: 0.59015
Precision: 0.5947637536333354
Recall: 0.5615809928282383
F Score: 0.5776962627896672


__Выводы__ в свободной форме:
Посчитали метрики, accuracy и остальные метрики довольно плохие из-за того, что находятся на уровне 60%, что почти 50%, а 50% - случайность. Пока что не увидели выигрыша от нормализации.

## Часть 2. Работа с категориальными переменными

В этой части мы научимся обрабатывать категориальные переменные, так как закодировать их в виде чисел недостаточно (это задаёт некоторый порядок, которого на категориальных переменных может и не быть). Существует два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

__Задание 1.__ Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (с тем, что было ранее). Измерьте время, потребовавшееся на обучение модели.

__(3 балла)__

In [16]:
# data.hist()
data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,1.366550,4.483870,0.429490,0.502265,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.674421,2.739255,0.496689,1.501934,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,2.000000,7.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
from sklearn.preprocessing import OneHotEncoder
categorical_features = [c for c in data.columns if c.endswith('_cat')]

In [18]:
encoder = OneHotEncoder(sparse_output=False, drop="first")
encoded_features = encoder.fit_transform(data[categorical_features])

data_one_hot = data.drop(categorical_features, axis=1)
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

data_one_hot.reset_index(inplace=True, drop=True)
data_one_hot = pd.concat([data_one_hot, encoded_df], axis=1)

X_train, X_test, y_train, y_test = train_test_split(normalize(data_one_hot), target, test_size=0.5)

train_model(X_train, X_test, y_train, y_test)

Accuracy: 0.59303
Precision: 0.5981266726137378
Recall: 0.5639305524961451
F Score: 0.5805254640843546


In [19]:
data_one_hot.describe()

,ps_ind_01,ps_ind_03,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,...,ps_car_11_cat_95,ps_car_11_cat_96,ps_car_11_cat_97,ps_car_11_cat_98,ps_car_11_cat_99,ps_car_11_cat_100,ps_car_11_cat_101,ps_car_11_cat_102,ps_car_11_cat_103,ps_car_11_cat_104
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.00000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,4.483870,0.351100,0.295375,0.175920,0.177605,0.000470,0.001925,0.011675,0.001090,...,0.005090,0.003105,0.00319,0.004255,0.016775,0.009200,0.012870,0.003385,0.035640,0.159235
std,2.017199,2.739255,0.477315,0.456212,0.380753,0.382181,0.021674,0.043833,0.107419,0.032997,...,0.071163,0.055636,0.05639,0.065092,0.128428,0.095475,0.112714,0.058082,0.185391,0.365896
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,7.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,11.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Как можно было заменить, one-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирование категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

__Задание 2.__ Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве. Сравните время обучения с предыдущим экспериментов. Заметили ли вы что-то интересное?

__(2 балла)__

In [20]:
data_cat_cnt = data.copy()


for feature in categorical_features:
    for i in np.unique(data_cat_cnt[feature]):
        data_cat_cnt[feature][data_cat_cnt[feature] == i] = np.mean(target[data_cat_cnt[feature] == i])


X_train, X_test, y_train, y_test = train_test_split(normalize(data_cat_cnt), target, test_size=0.5)

train_model(X_train, X_test, y_train, y_test)

C:\Users\a9282\AppData\Local\Temp\ipykernel_2596\3437435449.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cat_cnt[feature][data_cat_cnt[feature] == i] = np.mean(target[data_cat_cnt[feature] == i])
C:\Users\a9282\AppData\Local\Temp\ipykernel_2596\3437435449.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9112149532710281' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_cat_cnt[feature][data_cat_cnt[feature] == i] = np.mean(target[data_cat_cnt[feature] == i])
C:\Users\a9282\AppData\Local\Temp\ipykernel_2596\3437435449.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

Accuracy: 0.59573
Precision: 0.6008200012680432
Recall: 0.5689441242395132
F Score: 0.5844477565914581


In [21]:
data_cat_cnt.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.500000,4.483870,0.500000,0.500000,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.015959,2.739255,0.020419,0.050252,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.494915,0.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.494915,2.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.494915,4.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.500783,7.000000,0.517318,0.480547,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,0.911215,11.000000,0.964072,0.697423,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


__Вывод:__ Ну видимо в данной задачи one-hot кодирование ведет себя немного хуже, чем счетчики, так как категориальные переменные не так важны по отдельности в этой задачи и не стоит выделять 100+ 0/1 признаков в датасет, которые не сильно влияют на страховку.

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:
- вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
- вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
- внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

__Задание 3.__ Реализуйте корректное вычисление счётчиков двумя из трех вышеперчисленных способов, сравните. Снова обучите логистическую регрессию, оцените качество. Сделайте выводы.

__(3 балла)__

In [30]:
# ПОШУМИМ НЕМНОГО ;)
data_noised = data_cat_cnt.copy()
categorical_features = [c for c in data.columns if c.endswith('_cat')]

for feature in categorical_features:
    uniq = np.unique(data[feature])
    for value in uniq:
        current = np.array(target[data[feature] == value])
        data_noised[feature][data[feature] == value] = np.mean(current) + np.random.random(current.shape) * 0.01

X_train, X_test, y_train, y_test = train_test_split(normalize(data_noised), target, test_size=0.5)

train_model(X_train, X_test, y_train, y_test)

C:\Users\a9282\AppData\Local\Temp\ipykernel_2596\2970628011.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_noised[feature][data[feature] == value] = np.mean(current) + np.random.random(current.shape) * 0.01
C:\Users\a9282\AppData\Local\Temp\ipykernel_2596\2970628011.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_noised[feature][data[feature] == value] = np.mean(current) + np.random.random(current.shape) * 0.01
C:\Users\a9282\AppData\Local\Temp\ipykernel_2596\2970628011.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentati

Accuracy: 0.59497
Precision: 0.597323702559392
Recall: 0.5742468942541192
F Score: 0.5855580226954128


__Вывод:__ Вдарили рок в этой дыре, но значимых результатов от этого достигнуто не было, хотя теоретически так делать более правильно

## Часть 2. Метод опорных векторов и калибровка вероятностней

__Задание 1.__ Обучение и применение метода опорных векторов.

__(1 балл)__

Обучите метод опорных векторов (воспользуйтесь готовой реализацией LinearSVC из sklearn). Используйте уже загруженные и обработанные в предыдущей части данные.

In [33]:
from sklearn.svm import LinearSVC

svc = LinearSVC(random_state=0) 
X_train, X_test, y_train, y_test = train_test_split(normalize(data_noised), target, test_size=0.5)

svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)

c:\Users\a9282\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


На той же тестовой части посчитайте все те же метрики. Что вы можете сказать о полученных результатах?

In [34]:
accuracysvc = accuracy_score(y_test, y_pred)
precisionsvc = precision_score(y_test, y_pred)
recallsvc = recall_score(y_test, y_pred)
f1svc = f1_score(y_test, y_pred)
print(f"Accuracy:", accuracysvc)
print(f"Precision:", precisionsvc)
print(f"Recall:", recallsvc)
print(f"F1 Score:", f1svc)

Accuracy: 0.59225
Precision: 0.5976619165457804
Recall: 0.5610943101480043
F1 Score: 0.5788011197537368



__Вывод:__ Самое главное, что вынес с домашки - Kaggle не такой страшный сайт, как казалось раньше. Почитали про метрики, но конкретно в данном примере все метрики дают почти одинаковые значения.